# 🏥 Notebook 4: AI Engineering

In this final notebook, we move beyond training into the **full lifecycle of an AI product**.  
This includes **deployment, monitoring, and maintenance**.  

---

## 🎯 Learning Objectives
By the end of this notebook, you will be able to:
1. Explain how to deploy an ML model using **FastAPI**.
2. Understand how to track experiments and monitor drift with **MLflow**.
3. Describe strategies for **automatic retraining**.
4. Reflect on challenges of AI in healthcare deployment.  

---


## 4.1 Deploying with FastAPI

Once trained, a model needs to be **accessible** to apps, clinicians, or hospital systems.  
We achieve this by wrapping the model in a web API.  

👉 **FastAPI** is a modern, Pythonic framework for building REST APIs.

⚠️ Note: Running FastAPI fully requires a server (not just Colab).  
Here, we’ll show the *concept* with code snippets.


In [ ]:
#@title Install Dependencies (FastAPI demo) { display-mode: "form" }
!pip -q install fastapi uvicorn "python-multipart" scikit-learn joblib
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

# Train a tiny demo model
iris = load_iris()
X, y = iris.data, iris.target
clf = LogisticRegression(max_iter=200).fit(X, y)
joblib.dump(clf, "model.joblib")
print("✅ Demo model trained & saved (iris dataset)")


In [ ]:
#@title FastAPI App Example (saved as app.py)
%%writefile app.py
import joblib
from fastapi import FastAPI
from pydantic import BaseModel

# Load model
model = joblib.load("model.joblib")

# Define input schema
class Features(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

app = FastAPI(title="Demo AI Model API")

@app.get("/")
def read_root():
    return {"message": "Welcome to the AI Model API!"}

@app.post("/predict/")
def predict(features: Features):
    data = [[
        features.sepal_length,
        features.sepal_width,
        features.petal_length,
        features.petal_width
    ]]
    prediction = model.predict(data)[0]
    return {"prediction": int(prediction)}


💡 **How this works**:
- `/predict/` endpoint receives JSON patient features.
- FastAPI validates input → calls model → returns prediction.  

📌 To run locally:  
```bash
uvicorn app:app --reload
```


### 📊 4.2 Model Monitoring with MLflow  

```markdown
## 4.2 Model Monitoring with MLflow

After deployment, we must **track performance** over time.  
- **Experiment tracking:** log hyperparams, metrics, artifacts  
- **Model registry:** manage versions (v1, v2…)  
- **Monitoring drift:** detect if performance declines on new patient data

In [ ]:
#@title MLflow Demo (Logging Run)
!pip -q install mlflow

import mlflow
import numpy as np
from sklearn.metrics import accuracy_score

# Mock training
X_train, y_train = X, y
y_pred = clf.predict(X_train)
acc = accuracy_score(y_train, y_pred)

with mlflow.start_run():
    mlflow.log_param("model_type", "Logistic Regression")
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(clf, "model")

print("✅ Logged run to MLflow (local tracking server needed for UI)")


## 4.3 Automatic Retraining

Models can degrade over time (called **drift**) due to:
- Changing patient populations
- New hospital equipment
- Data recording practices

**Solution:** retrain periodically or when metrics drop below a threshold.  

⚠️ In production, this would be done with:
- ML pipelines (Airflow, Kubeflow)
- Continuous retraining jobs
- Human-in-the-loop for validation


In [ ]:
#@title Pseudo-code for Retraining Trigger
performance = 0.70  # Mock accuracy
threshold = 0.75

if performance < threshold:
    print("⚠️ Performance below threshold. Triggering retraining pipeline...")
    # In real setup: fetch new data → retrain model → log with MLflow → redeploy
else:
    print("✅ Performance acceptable. No retraining required.")


# ✅ Summary

In this notebook, you learned:
- How to wrap a model in a FastAPI service.
- How to log experiments & metrics with MLflow.
- Why automatic retraining is critical.

---

## 🌍 Reflection
1. What risks arise if a deployed healthcare model drifts unnoticed?  
2. Who should approve retrained models before deployment?  
3. How would you design a **safe retraining policy** in a hospital?  

---

🎉 Congratulations! You’ve completed the course:
- **Notebook 1:** Setup & Python Basics  
- **Notebook 2:** ML for Pneumonia Detection  
- **Notebook 3:** ML with Tabular Data  
- **Notebook 4:** AI Engineering & Deployment  

👉 Next steps: Build your **own healthcare AI project**, from data → model → deployment.
